# Temat testów:
Czy istnieje istotna statystycznie różnica w poprawności/pewności/czasie odpowiedzi w zalezności od cech osobowościowych z ankiety.

In [1]:
from scipy import stats
import pandas as pd
import statsmodels.api as sm

In [2]:
df_answers = pd.read_csv(
    '/Users/martasolarz/Studies/Thesis/data/correctness.csv',
    usecols=['ID', '1a_corr', '1b_corr', '2a_corr', '2b_corr', '3a_corr', '3b_corr', 'Wynik', '1a_trud', '1b_trud', '2a_trud', '2b_trud', '3a_trud', '3b_trud', 'Trud_all']
).fillna('NO_VAL')

df_quest = pd.read_csv(
    '/Users/martasolarz/Studies/Thesis/data/survey.csv',
).fillna('NO_VAL')

In [5]:
# zagregowane kolumny - podział na tych co pisali i nie pisali danej matury:
df_quest['MAT'] = df_quest['MAT_roz'].apply(lambda x: x if x == 'NO_VAL' else 'VAL' )
df_quest['POL'] = df_quest['JP_roz'].apply(lambda x: x if x == 'NO_VAL' else 'VAL' )
df_quest['HIST'] = df_quest['HIST_roz'].apply(lambda x: x if x == 'NO_VAL' else 'VAL' )

df_quest.head()

,ID,Płeć,Rok_urodzenia,Kierunek_studiów,Rok_studiów,Specjalność,Wady_wzroku,Daltonizm,Ilość_snu,JP_podst,...,Quest2_after,Rank_map,Rank_txt,Rank_plt,Rank_tab,Data,Godzina,MAT,POL,HIST
0,1,M,2002,"Geografia, w ramach studiów międzyobszarowych",2,SPOŁ-EKO,0,1,6,90 - 100%,...,wykres,3,4,1,2,2023-05-24,11:44:20,VAL,VAL,NO_VAL
1,2,M,2003,Gospodarka przestrzenna,1,NO_VAL,1,1,<6,50 - 69%,...,wykres,2,3,1,4,2023-05-24,12:21:03,VAL,VAL,NO_VAL
2,3,M,2001,Geografia,3,GEOINF,0,1,8,50 - 69%,...,mapa,1,4,3,2,2023-05-24,12:41:09,VAL,NO_VAL,NO_VAL
3,4,K,2000,Geografia,3,GEOINF,0,1,8,50 - 69%,...,wykres,2,4,3,1,2023-05-24,13:15:20,NO_VAL,VAL,NO_VAL
4,5,K,2002,Geografia,3,GEOINF,0,1,7,50 - 69%,...,tabela,2,4,1,3,2023-05-24,13:35:17,NO_VAL,NO_VAL,NO_VAL


In [6]:
def choose_dominant_kind(x):
    k = x['Kinestetyk']
    s = x['Słuchowiec']
    w = x['Wzrokowiec']
    if k > s and k > w:
        return 'Kinestetyk'
    elif s > k and s > w:
        return 'Słuchowiec'
    elif w > k and w > s:
        return 'Wzrokowiec'
    else:
        return "Brak dominującego"

In [8]:
df_quest['Typ_uczenia'] = df_quest.apply(choose_dominant_kind, axis=1)
df_quest.head()

,ID,Płeć,Rok_urodzenia,Kierunek_studiów,Rok_studiów,Specjalność,Wady_wzroku,Daltonizm,Ilość_snu,JP_podst,...,Rank_map,Rank_txt,Rank_plt,Rank_tab,Data,Godzina,MAT,POL,HIST,Typ_uczenia
0,1,M,2002,"Geografia, w ramach studiów międzyobszarowych",2,SPOŁ-EKO,0,1,6,90 - 100%,...,3,4,1,2,2023-05-24,11:44:20,VAL,VAL,NO_VAL,Kinestetyk
1,2,M,2003,Gospodarka przestrzenna,1,NO_VAL,1,1,<6,50 - 69%,...,2,3,1,4,2023-05-24,12:21:03,VAL,VAL,NO_VAL,Brak dominującego
2,3,M,2001,Geografia,3,GEOINF,0,1,8,50 - 69%,...,1,4,3,2,2023-05-24,12:41:09,VAL,NO_VAL,NO_VAL,Wzrokowiec
3,4,K,2000,Geografia,3,GEOINF,0,1,8,50 - 69%,...,2,4,3,1,2023-05-24,13:15:20,NO_VAL,VAL,NO_VAL,Brak dominującego
4,5,K,2002,Geografia,3,GEOINF,0,1,7,50 - 69%,...,2,4,1,3,2023-05-24,13:35:17,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec


In [10]:
df_merge = df_answers.merge(df_quest, on='ID').set_index('ID')
df_merge.tail()

,1a_trud,1b_trud,2a_trud,2b_trud,3a_trud,3b_trud,Trud_all,Wynik,1a_corr,1b_corr,...,Rank_map,Rank_txt,Rank_plt,Rank_tab,Data,Godzina,MAT,POL,HIST,Typ_uczenia
ID,,,,,,,,,,,,,,,,,,,,,
42,2,1,2,1,2,2,10,5,1,1,...,2,4,1,3,2023-06-07,11:42:55,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec
43,1,2,2,2,3,3,13,5,1,1,...,2,3,1,2,2023-06-07,13:04:51,NO_VAL,NO_VAL,NO_VAL,Brak dominującego
44,1,1,3,1,2,3,11,3,1,1,...,2,1,2,1,2023-06-07,14:13:38,VAL,NO_VAL,NO_VAL,Brak dominującego
45,2,1,2,2,2,2,11,3,1,1,...,2,4,3,1,2023-06-07,14:27:37,VAL,NO_VAL,NO_VAL,Słuchowiec
46,1,3,2,2,2,3,13,4,0,1,...,1,3,3,3,2023-06-07,15:19:52,NO_VAL,NO_VAL,NO_VAL,Brak dominującego


Funkcje potrzebne do testów:

In [11]:
def prepate_col(list_of_cols_popr):
    new_row_popr = list_of_cols_popr[0]
    for i in list_of_cols_popr[1:]:
        new_row_popr = pd.concat([new_row_popr, i], ignore_index=True)

    return new_row_popr

In [12]:
def make_decision(p_val, alpha):
    if p_val < alpha:
        print('Istnieją podstawy do odrzucenia H0, przyjmujemy hipotezę H1:')
    else:
        print('Brak podstaw do odrzucenia H0.')
        
def conduct_chi2test(contingency_table, alpha):
    chi2_stat, p_val, _, _ = stats.chi2_contingency(contingency_table)

    print('Wartość statystyki chi-kwadrat:', chi2_stat)
    print('P-value:', p_val)
    make_decision(p_val, alpha)

In [13]:
def test_parametr(dfs, names, dependent_variable, independent_variable):
    for i, df in enumerate(dfs):
        print('-----------------------------------------------')
        print(names[i+1])
        tab = pd.crosstab(df[dependent_variable], df[independent_variable])
        print(tab)
        conduct_chi2test(tab, ALPHA)

In [14]:
def test_posthoc(df, dependent_variable, independent_variable):
    posthoc = sm.stats.multicomp.MultiComparison(df[dependent_variable], df[independent_variable])
    result = posthoc.tukeyhsd()
    print(result)

Przygotowanie data setów dla wszystkich testowanych przypadków:

1. Poprawność vs dane osobowe
2. Pewność vs dane osobowe

In [20]:
# zsumowany wynik
df_wynik = df_merge.drop(columns=['1a_corr', '1b_corr', '2a_corr', '2b_corr', '3a_corr', '3b_corr', '1a_trud', '1b_trud', '2a_trud', '2b_trud', '3a_trud', '3b_trud'])
df_wynik.rename(columns={'Wynik':'corr', 'Trud_all': 'trud'}, inplace=True)
df_wynik

,trud,corr,Płeć,Rok_urodzenia,Kierunek_studiów,Rok_studiów,Specjalność,Wady_wzroku,Daltonizm,Ilość_snu,...,Rank_map,Rank_txt,Rank_plt,Rank_tab,Data,Godzina,MAT,POL,HIST,Typ_uczenia
ID,,,,,,,,,,,,,,,,,,,,,
1,19,4,M,2002,"Geografia, w ramach studiów międzyobszarowych",2,SPOŁ-EKO,0,1,6,...,3,4,1,2,2023-05-24,11:44:20,VAL,VAL,NO_VAL,Kinestetyk
2,13,5,M,2003,Gospodarka przestrzenna,1,NO_VAL,1,1,<6,...,2,3,1,4,2023-05-24,12:21:03,VAL,VAL,NO_VAL,Brak dominującego
3,9,4,M,2001,Geografia,3,GEOINF,0,1,8,...,1,4,3,2,2023-05-24,12:41:09,VAL,NO_VAL,NO_VAL,Wzrokowiec
4,16,5,K,2000,Geografia,3,GEOINF,0,1,8,...,2,4,3,1,2023-05-24,13:15:20,NO_VAL,VAL,NO_VAL,Brak dominującego
5,17,6,K,2002,Geografia,3,GEOINF,0,1,7,...,2,4,1,3,2023-05-24,13:35:17,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec
6,19,5,M,2003,Geografia,1,FIZ,1,1,6,...,1,4,2,2,2023-05-24,15:42:35,VAL,NO_VAL,NO_VAL,Wzrokowiec
7,14,6,K,2001,"Geografia, w ramach studiów międzyobszarowych",3,GEOINF,0,1,7,...,2,4,1,3,2023-05-25,13:46:15,VAL,NO_VAL,VAL,Brak dominującego
11,11,2,M,2003,Geografia,1,GEOINF,0,1,8<,...,1,4,1,3,2023-05-25,14:12:48,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec
12,12,2,K,2003,Geografia,1,FIZ,0,1,8<,...,1,4,2,3,2023-05-25,14:47:58,VAL,NO_VAL,NO_VAL,Brak dominującego


In [21]:
df_list = [df_quest] * 6
df_all = pd.concat(df_list, ignore_index=True)

popr = []
for col in df_answers.columns.tolist():
    if col.endswith('corr'):
        popr.append(df_answers[col])
colPopr = prepate_col(popr)

trud = []
for col in df_answers.columns.tolist():
    if col.endswith('trud'):
        trud.append(df_answers[col])
colTrud = prepate_col(trud)

df_all["corr"] = colPopr
df_all["trud"] = colTrud

df_all

,ID,Płeć,Rok_urodzenia,Kierunek_studiów,Rok_studiów,Specjalność,Wady_wzroku,Daltonizm,Ilość_snu,JP_podst,...,Rank_plt,Rank_tab,Data,Godzina,MAT,POL,HIST,Typ_uczenia,corr,trud
0,1,M,2002,"Geografia, w ramach studiów międzyobszarowych",2,SPOŁ-EKO,0,1,6,90 - 100%,...,1,2,2023-05-24,11:44:20,VAL,VAL,NO_VAL,Kinestetyk,1,3
1,2,M,2003,Gospodarka przestrzenna,1,NO_VAL,1,1,<6,50 - 69%,...,1,4,2023-05-24,12:21:03,VAL,VAL,NO_VAL,Brak dominującego,1,1
2,3,M,2001,Geografia,3,GEOINF,0,1,8,50 - 69%,...,3,2,2023-05-24,12:41:09,VAL,NO_VAL,NO_VAL,Wzrokowiec,1,2
3,4,K,2000,Geografia,3,GEOINF,0,1,8,50 - 69%,...,3,1,2023-05-24,13:15:20,NO_VAL,VAL,NO_VAL,Brak dominującego,1,3
4,5,K,2002,Geografia,3,GEOINF,0,1,7,50 - 69%,...,1,3,2023-05-24,13:35:17,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,42,K,2003,Geografia,1,FIZ,0,1,7,70 - 89%,...,1,3,2023-06-07,11:42:55,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec,1,2
242,43,M,2003,Gospodarka przestrzenna,1,NO_VAL,1,1,8,50 - 69%,...,1,2,2023-06-07,13:04:51,NO_VAL,NO_VAL,NO_VAL,Brak dominującego,0,3
243,44,K,2003,Geografia,1,FIZ,1,1,<6,50 - 69%,...,2,1,2023-06-07,14:13:38,VAL,NO_VAL,NO_VAL,Brak dominującego,0,3
244,45,M,2003,Geografia,1,GEOINF,1,1,8<,30 - 49%,...,3,1,2023-06-07,14:27:37,VAL,NO_VAL,NO_VAL,Słuchowiec,0,2


In [22]:
# grafiki A
df_list = [df_quest] * 3
df_a = pd.concat(df_list, ignore_index=True)

popr = []
for col in df_answers.columns.tolist():
    if col.endswith('a_corr'):
        popr.append(df_answers[col])
colPopr = prepate_col(popr)

trud = []
for col in df_answers.columns.tolist():
    if col.endswith('a_trud'):
        trud.append(df_answers[col])
colTrud = prepate_col(trud)

df_a["corr"] = colPopr
df_a["trud"] = colTrud
df_a

,ID,Płeć,Rok_urodzenia,Kierunek_studiów,Rok_studiów,Specjalność,Wady_wzroku,Daltonizm,Ilość_snu,JP_podst,...,Rank_plt,Rank_tab,Data,Godzina,MAT,POL,HIST,Typ_uczenia,corr,trud
0,1,M,2002,"Geografia, w ramach studiów międzyobszarowych",2,SPOŁ-EKO,0,1,6,90 - 100%,...,1,2,2023-05-24,11:44:20,VAL,VAL,NO_VAL,Kinestetyk,1,3
1,2,M,2003,Gospodarka przestrzenna,1,NO_VAL,1,1,<6,50 - 69%,...,1,4,2023-05-24,12:21:03,VAL,VAL,NO_VAL,Brak dominującego,1,1
2,3,M,2001,Geografia,3,GEOINF,0,1,8,50 - 69%,...,3,2,2023-05-24,12:41:09,VAL,NO_VAL,NO_VAL,Wzrokowiec,1,2
3,4,K,2000,Geografia,3,GEOINF,0,1,8,50 - 69%,...,3,1,2023-05-24,13:15:20,NO_VAL,VAL,NO_VAL,Brak dominującego,1,3
4,5,K,2002,Geografia,3,GEOINF,0,1,7,50 - 69%,...,1,3,2023-05-24,13:35:17,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,42,K,2003,Geografia,1,FIZ,0,1,7,70 - 89%,...,1,3,2023-06-07,11:42:55,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec,0,2
119,43,M,2003,Gospodarka przestrzenna,1,NO_VAL,1,1,8,50 - 69%,...,1,2,2023-06-07,13:04:51,NO_VAL,NO_VAL,NO_VAL,Brak dominującego,1,3
120,44,K,2003,Geografia,1,FIZ,1,1,<6,50 - 69%,...,2,1,2023-06-07,14:13:38,VAL,NO_VAL,NO_VAL,Brak dominującego,0,2
121,45,M,2003,Geografia,1,GEOINF,1,1,8<,30 - 49%,...,3,1,2023-06-07,14:27:37,VAL,NO_VAL,NO_VAL,Słuchowiec,1,2


In [23]:
# grafiki B
df_list = [df_quest] * 3
df_b = pd.concat(df_list, ignore_index=True)

popr = []
for col in df_answers.columns.tolist():
    if col.endswith('b_corr'):
        popr.append(df_answers[col])
colPopr = prepate_col(popr)

trud = []
for col in df_answers.columns.tolist():
    if col.endswith('b_trud'):
        trud.append(df_answers[col])
colTrud = prepate_col(trud)

df_b["corr"] = colPopr
df_b["trud"] = colTrud
df_b

,ID,Płeć,Rok_urodzenia,Kierunek_studiów,Rok_studiów,Specjalność,Wady_wzroku,Daltonizm,Ilość_snu,JP_podst,...,Rank_plt,Rank_tab,Data,Godzina,MAT,POL,HIST,Typ_uczenia,corr,trud
0,1,M,2002,"Geografia, w ramach studiów międzyobszarowych",2,SPOŁ-EKO,0,1,6,90 - 100%,...,1,2,2023-05-24,11:44:20,VAL,VAL,NO_VAL,Kinestetyk,1,1
1,2,M,2003,Gospodarka przestrzenna,1,NO_VAL,1,1,<6,50 - 69%,...,1,4,2023-05-24,12:21:03,VAL,VAL,NO_VAL,Brak dominującego,1,2
2,3,M,2001,Geografia,3,GEOINF,0,1,8,50 - 69%,...,3,2,2023-05-24,12:41:09,VAL,NO_VAL,NO_VAL,Wzrokowiec,1,1
3,4,K,2000,Geografia,3,GEOINF,0,1,8,50 - 69%,...,3,1,2023-05-24,13:15:20,NO_VAL,VAL,NO_VAL,Brak dominującego,1,2
4,5,K,2002,Geografia,3,GEOINF,0,1,7,50 - 69%,...,1,3,2023-05-24,13:35:17,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,42,K,2003,Geografia,1,FIZ,0,1,7,70 - 89%,...,1,3,2023-06-07,11:42:55,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec,1,2
119,43,M,2003,Gospodarka przestrzenna,1,NO_VAL,1,1,8,50 - 69%,...,1,2,2023-06-07,13:04:51,NO_VAL,NO_VAL,NO_VAL,Brak dominującego,0,3
120,44,K,2003,Geografia,1,FIZ,1,1,<6,50 - 69%,...,2,1,2023-06-07,14:13:38,VAL,NO_VAL,NO_VAL,Brak dominującego,0,3
121,45,M,2003,Geografia,1,GEOINF,1,1,8<,30 - 49%,...,3,1,2023-06-07,14:27:37,VAL,NO_VAL,NO_VAL,Słuchowiec,0,2


In [40]:
# grafiki 1,2,3
df_list = [df_quest] * 2
df_1 = pd.concat(df_list, ignore_index=True)

popr1 = []
for col in df_answers.columns.tolist():
    if col.startswith('1') and col.endswith('_corr'):
        popr1.append(df_answers[col])
colPopr = prepate_col(popr1)

trud1 = []
for col in df_answers.columns.tolist():
    if col.startswith('1') and col.endswith('_trud'):
        trud1.append(df_answers[col])
colTrud = prepate_col(trud1)

df_1["corr"] = colPopr
df_1["trud"] = colTrud

df_list = [df_quest] * 2
df_2 = pd.concat(df_list, ignore_index=True)

popr2 = []
for col in df_answers.columns.tolist():
    if col.startswith('2') and col.endswith('_corr'):
        popr2.append(df_answers[col])
colPopr = prepate_col(popr2)

trud2 = []
for col in df_answers.columns.tolist():
    if col.startswith('2') and col.endswith('_trud'):
        trud2.append(df_answers[col])
colTrud = prepate_col(trud2)

df_2["corr"] = colPopr
df_2["trud"] = colTrud

df_list = [df_quest] * 2
df_3 = pd.concat(df_list, ignore_index=True)

popr3 = []
for col in df_answers.columns.tolist():
    if col.startswith('3') and col.endswith('_corr'):
        popr3.append(df_answers[col])
colPopr = prepate_col(popr3)

trud3 = []
for col in df_answers.columns.tolist():
    if col.startswith('3') and col.endswith('_trud'):
        trud3.append(df_answers[col])
colTrud = prepate_col(trud3)

df_3["corr"] = colPopr
df_3["trud"] = colTrud

df_2

,ID,Płeć,Rok_urodzenia,Kierunek_studiów,Rok_studiów,Specjalność,Wady_wzroku,Daltonizm,Ilość_snu,JP_podst,...,Rank_plt,Rank_tab,Data,Godzina,MAT,POL,HIST,Typ_uczenia,corr,trud
0,1,M,2002,"Geografia, w ramach studiów międzyobszarowych",2,SPOŁ-EKO,0,1,6,90 - 100%,...,1,2,2023-05-24,11:44:20,VAL,VAL,NO_VAL,Kinestetyk,0,4
1,2,M,2003,Gospodarka przestrzenna,1,NO_VAL,1,1,<6,50 - 69%,...,1,4,2023-05-24,12:21:03,VAL,VAL,NO_VAL,Brak dominującego,1,3
2,3,M,2001,Geografia,3,GEOINF,0,1,8,50 - 69%,...,3,2,2023-05-24,12:41:09,VAL,NO_VAL,NO_VAL,Wzrokowiec,1,3
3,4,K,2000,Geografia,3,GEOINF,0,1,8,50 - 69%,...,3,1,2023-05-24,13:15:20,NO_VAL,VAL,NO_VAL,Brak dominującego,1,2
4,5,K,2002,Geografia,3,GEOINF,0,1,7,50 - 69%,...,1,3,2023-05-24,13:35:17,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,42,K,2003,Geografia,1,FIZ,0,1,7,70 - 89%,...,1,3,2023-06-07,11:42:55,NO_VAL,NO_VAL,NO_VAL,Wzrokowiec,1,1
78,43,M,2003,Gospodarka przestrzenna,1,NO_VAL,1,1,8,50 - 69%,...,1,2,2023-06-07,13:04:51,NO_VAL,NO_VAL,NO_VAL,Brak dominującego,1,2
79,44,K,2003,Geografia,1,FIZ,1,1,<6,50 - 69%,...,2,1,2023-06-07,14:13:38,VAL,NO_VAL,NO_VAL,Brak dominującego,1,1
80,45,M,2003,Geografia,1,GEOINF,1,1,8<,30 - 49%,...,3,1,2023-06-07,14:27:37,VAL,NO_VAL,NO_VAL,Słuchowiec,0,2


**H0:** Nie ma istotnej statystycznie zależności między poprawnością/trudnością odpowiedzi a wartością cechy X.
**H1:** Istnieje istotna statystycznie zależność między poprawnością/trudnością odpowiedzi a wartością cechy X.
**Poziom istotności:** $\alpha=0.05$

In [41]:
ALPHA = 0.05

In [42]:
df_quest.columns

Index(['ID', 'Płeć', 'Rok_urodzenia', 'Kierunek_studiów', 'Rok_studiów',
       'Specjalność', 'Wady_wzroku', 'Daltonizm', 'Ilość_snu', 'JP_podst',
       'MAT_podst', 'JP_roz', 'MAT_roz', 'FIZ_roz', 'HIST_roz', 'Samopocz',
       'Stres', 'Zmęcz', 'Czyt_map', 'Podziel_uwg', 'Analit_myśl', 'Spostrzeg',
       'Czyt_ze_zroz', 'Wzrokowiec', 'Słuchowiec', 'Kinestetyk',
       'Quest1_after', 'Quest2_after', 'Rank_map', 'Rank_txt', 'Rank_plt',
       'Rank_tab', 'Data', 'Godzina', 'MAT', 'POL', 'HIST', 'Typ_uczenia'],
      dtype='object')

In [43]:
dfs = [df_wynik, df_all, df_a, df_b, df_1, df_2, df_3]
names = {
    1: 'Zsumowany wynik:',
    2: 'Wszystkie grafiki:',
    3: 'Grafiki A:',
    4: 'Grafiki B:',
    5: 'Grafiki 1:',
    6: 'Grafiki 2:',
    7: 'Grafiki 3:',
}

### TESTY:

Poprawność

In [44]:
# płeć
test_parametr(dfs, names, 'corr', 'Płeć')

-----------------------------------------------
Zsumowany wynik:
Płeć  K   M
corr       
0     1   0
2     2   1
3     6   2
4     5   3
5     4  10
6     5   2
Wartość statystyki chi-kwadrat: 7.1876150218541515
P-value: 0.20705689729289914
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Płeć   K   M
corr        
0     46  26
1     92  82
Wartość statystyki chi-kwadrat: 2.081714061950506
P-value: 0.149072696245845
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Płeć   K   M
corr        
0     24  12
1     45  42
Wartość statystyki chi-kwadrat: 1.741652900401651
P-value: 0.1869295295546119
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki B:
Płeć   K   M
corr        
0     22  14
1     47  40
Wartość statystyki chi-kwadrat: 0.27151297036666844
P-value: 0.6023186601021169
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki 1:


In [45]:
# rok urodzenia
test_parametr(dfs, names, 'corr', 'Rok_urodzenia')

-----------------------------------------------
Zsumowany wynik:
Rok_urodzenia  2000  2001  2002  2003  2004
corr                                       
0                 0     1     0     0     0
2                 0     0     1     2     0
3                 0     3     1     4     0
4                 0     1     5     2     0
5                 1     0     3     9     1
6                 0     3     1     1     2
Wartość statystyki chi-kwadrat: 27.3549031986532
P-value: 0.12557108742908202
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Rok_urodzenia  2000  2001  2002  2003  2004
corr                                       
0                 1    17    20    33     1
1                 5    31    46    75    17
Wartość statystyki chi-kwadrat: 6.346375029025891
P-value: 0.17473104399734973
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Rok_urodzenia  2000  2001  2002  2003  2004
corr              

In [46]:
test_posthoc(df_1, 'corr', 'Rok_urodzenia')

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
  2000   2001  -0.3125  0.592 -0.9109 0.2859  False
  2000   2002  -0.0455 0.9995 -0.6347 0.5438  False
  2000   2003  -0.0556 0.9988 -0.6352 0.5241  False
  2000   2004      0.0    1.0 -0.6514 0.6514  False
  2001   2002    0.267 0.0437  0.0049 0.5292   True
  2001   2003   0.2569 0.0295  0.0172 0.4967   True
  2001   2004   0.3125 0.1607 -0.0694 0.6944  False
  2002   2003  -0.0101 0.9999  -0.226 0.2058  False
  2002   2004   0.0455 0.9969  -0.322 0.4129  False
  2003   2004   0.0556  0.992 -0.2963 0.4074  False
---------------------------------------------------


In [47]:
# kierunek studiów
test_parametr(dfs, names, 'corr', 'Kierunek_studiów')

-----------------------------------------------
Zsumowany wynik:
Kierunek_studiów  Geografia  Geografia, w ramach studiów międzyobszarowych  \
corr                                                                         
0                         1                                              0   
2                         2                                              0   
3                         7                                              0   
4                         7                                              1   
5                        10                                              1   
6                         6                                              1   

Kierunek_studiów  Gospodarka przestrzenna  
corr                                       
0                                       0  
2                                       1  
3                                       1  
4                                       0  
5                                       3  
6 

In [48]:
test_posthoc(df_3, 'corr', 'Kierunek_studiów')

                                       Multiple Comparison of Means - Tukey HSD, FWER=0.05                                       
                    group1                                        group2                    meandiff p-adj   lower  upper  reject
---------------------------------------------------------------------------------------------------------------------------------
                                    Geografia Geografia, w ramach studiów międzyobszarowych   0.5303 0.0339  0.0329 1.0277   True
                                    Geografia                       Gospodarka przestrzenna  -0.0697 0.9072 -0.4655 0.3261  False
Geografia, w ramach studiów międzyobszarowych                       Gospodarka przestrzenna     -0.6 0.0511 -1.2024 0.0024  False
---------------------------------------------------------------------------------------------------------------------------------


In [49]:
# rok studiów
test_parametr(dfs, names, 'corr', 'Rok_studiów')

-----------------------------------------------
Zsumowany wynik:
Rok_studiów   1  2  3
corr                 
0             1  0  0
2             3  0  0
3             4  1  3
4             3  4  1
5            10  3  1
6             3  0  4
Wartość statystyki chi-kwadrat: 16.188492063492063
P-value: 0.09436223810908975
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Rok_studiów   1   2   3
corr                   
0            46  14  12
1            98  34  42
Wartość statystyki chi-kwadrat: 1.7934227330779056
P-value: 0.40790891928772866
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Rok_studiów   1   2   3
corr                   
0            25   6   5
1            47  18  22
Wartość statystyki chi-kwadrat: 2.7528602596849723
P-value: 0.25247826079300134
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki B:
Rok_studiów   1   2   3
corr                   
0

In [50]:
# specjalność
test_parametr(dfs, names, 'corr', 'Specjalność')

-----------------------------------------------
Zsumowany wynik:
Specjalność  FIZ   GEOINF   NO_VAL  SPOŁ-EKO
corr                                        
0               1        0       0         0
2               1        1       1         0
3               1        5       1         1
4               1        3       0         4
5               5        4       3         2
6               1        5       0         1
Wartość statystyki chi-kwadrat: 16.69353505291005
P-value: 0.337512521148565
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Specjalność  FIZ   GEOINF   NO_VAL  SPOŁ-EKO
corr                                        
0              20       29      10        13
1              40       79      20        35
Wartość statystyki chi-kwadrat: 1.1337071626223914
P-value: 0.7689438968268528
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Specjalność  FIZ   GEOINF   NO_VAL  SPOŁ-EKO
corr  

In [51]:
# wady wzroku
test_parametr(dfs, names, 'corr', 'Wady_wzroku')

-----------------------------------------------
Zsumowany wynik:
Wady_wzroku  0   1
corr              
0            0   1
2            2   1
3            2   6
4            4   4
5            4  10
6            4   3
Wartość statystyki chi-kwadrat: 4.278154761904762
P-value: 0.5101021303841938
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Wady_wzroku   0    1
corr                
0            26   46
1            70  104
Wartość statystyki chi-kwadrat: 0.21060853687739414
P-value: 0.6462908566505332
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Wady_wzroku   0   1
corr               
0            12  24
1            36  51
Wartość statystyki chi-kwadrat: 0.3958866139846738
P-value: 0.5292212634953144
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki B:
Wady_wzroku   0   1
corr               
0            14  22
1            34  53
Wartość statystyki chi-k

In [52]:
# ilość snu
test_parametr(dfs, names, 'corr', 'Ilość_snu')

-----------------------------------------------
Zsumowany wynik:
Ilość_snu  6  7  8  8<  <6
corr                      
0          1  0  0   0   0
2          1  0  0   2   0
3          1  0  0   2   5
4          3  4  1   0   0
5          4  3  2   2   3
6          2  2  1   0   2
Wartość statystyki chi-kwadrat: 26.27172619047619
P-value: 0.15702399232133712
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Ilość_snu   6   7   8  8<  <6
corr                         
0          23  11   4  16  18
1          49  43  20  20  42
Wartość statystyki chi-kwadrat: 8.175891336739037
P-value: 0.08534345467127644
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Ilość_snu   6   7   8  8<  <6
corr                         
0          12   5   1   7  11
1          24  22  11  11  19
Wartość statystyki chi-kwadrat: 5.932912941677308
P-value: 0.20421470740433081
Brak podstaw do odrzucenia H0.
-----------------------

In [53]:
test_posthoc(df_2, 'corr', 'Ilość_snu')

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     6      7     0.25 0.3555 -0.1284 0.6284  False
     6      8   0.4167 0.1408 -0.0788 0.9121  False
     6     8<  -0.0833 0.9825 -0.5124 0.3457  False
     6     <6   0.2167 0.4724 -0.1508 0.5841  False
     7      8   0.1667  0.895  -0.349 0.6824  False
     7     8<  -0.3333 0.2486 -0.7856  0.119  False
     7     <6  -0.0333 0.9993 -0.4276  0.361  False
     8     8<     -0.5 0.0964 -1.0539 0.0539  False
     8     <6     -0.2 0.8058 -0.7077 0.3077  False
    8<     <6      0.3 0.3308 -0.1431 0.7431  False
---------------------------------------------------


In [54]:
# matura - j.pol - podst
test_parametr(dfs, names, 'corr', 'JP_podst')

-----------------------------------------------
Zsumowany wynik:
JP_podst  30 - 49%  50 - 69%  70 - 89%  90 - 100%
corr                                             
0                1         0         0          0
2                1         2         0          0
3                1         2         4          1
4                1         2         3          2
5                0         6         6          2
6                1         2         4          0
Wartość statystyki chi-kwadrat: 15.91108443377351
P-value: 0.38798556952908253
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
JP_podst  30 - 49%  50 - 69%  70 - 89%  90 - 100%
corr                                             
0               15        24        24          9
1               15        60        78         21
Wartość statystyki chi-kwadrat: 7.878629382787597
P-value: 0.04858781121195582
Istnieją podstawy do odrzucenia H0, przyjmujemy hipotezę H1:
------------------

In [55]:
test_posthoc(df_all, 'corr', 'JP_podst')

  Multiple Comparison of Means - Tukey HSD, FWER=0.05   
 group1    group2  meandiff p-adj   lower  upper  reject
--------------------------------------------------------
30 - 49%  50 - 69%   0.2143 0.1175 -0.0341 0.4626  False
30 - 49%  70 - 89%   0.2647 0.0263  0.0222 0.5072   True
30 - 49% 90 - 100%      0.2 0.3175 -0.1015 0.5015  False
50 - 69%  70 - 89%   0.0504 0.8731 -0.1216 0.2225  False
50 - 69% 90 - 100%  -0.0143 0.9988 -0.2626 0.2341  False
70 - 89% 90 - 100%  -0.0647 0.9007 -0.3072 0.1778  False
--------------------------------------------------------


In [56]:
# matura - mat - podst
test_parametr(dfs, names, 'corr', 'MAT_podst')

-----------------------------------------------
Zsumowany wynik:
MAT_podst  30 - 49%  50 - 69%  70 - 89%  90 - 100%
corr                                              
0                 0         0         1          0
2                 0         0         2          1
3                 0         1         1          6
4                 1         1         4          2
5                 0         0         4         10
6                 0         1         0          6
Wartość statystyki chi-kwadrat: 17.52668650793651
P-value: 0.28836537999378187
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
MAT_podst  30 - 49%  50 - 69%  70 - 89%  90 - 100%
corr                                              
0                 2         5        29         36
1                 4        13        43        114
Wartość statystyki chi-kwadrat: 6.293801085568329
P-value: 0.09815899164514029
Brak podstaw do odrzucenia H0.
------------------------------------

In [57]:
test_posthoc(df_3, 'corr', 'MAT_podst')

  Multiple Comparison of Means - Tukey HSD, FWER=0.05   
 group1    group2  meandiff p-adj   lower  upper  reject
--------------------------------------------------------
30 - 49%  50 - 69%   0.3333 0.8355 -0.7091 1.3758  False
30 - 49%  70 - 89%   0.3333 0.7881 -0.6063  1.273  False
30 - 49% 90 - 100%     0.62 0.2965 -0.3006 1.5406  False
50 - 69%  70 - 89%      0.0    1.0 -0.5827 0.5827  False
50 - 69% 90 - 100%   0.2867 0.5253 -0.2649 0.8383  False
70 - 89% 90 - 100%   0.2867 0.0907 -0.0304 0.6037  False
--------------------------------------------------------


In [58]:
# matura - j.pol - roz
test_parametr(dfs, names, 'corr', 'JP_roz')

-----------------------------------------------
Zsumowany wynik:
JP_roz  20 - 49%  50 - 70%  NO_VAL  powyżej 70%
corr                                           
0              0         0       1            0
2              0         0       3            0
3              0         1       6            1
4              0         1       5            2
5              1         3      10            0
6              0         0       7            0
Wartość statystyki chi-kwadrat: 10.806733630952381
P-value: 0.7661704957717559
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
JP_roz  20 - 49%  50 - 70%  NO_VAL  powyżej 70%
corr                                           
0              1         8      56            7
1              5        22     136           11
Wartość statystyki chi-kwadrat: 1.3640485312899107
P-value: 0.7139831630446276
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
JP_roz  20 - 

In [59]:
# matura - mat - roz
test_parametr(dfs, names, 'corr', 'MAT_roz')

-----------------------------------------------
Zsumowany wynik:
MAT_roz  20 - 49%  50 - 70%  NO_VAL  poniżej 20%  powyżej 70%
corr                                                         
0               0         0       1            0            0
2               0         1       1            1            0
3               0         2       3            2            1
4               0         2       4            1            1
5               2         4       6            0            2
6               0         2       3            1            1
Wartość statystyki chi-kwadrat: 9.751304713804712
P-value: 0.9724605087534033
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
MAT_roz  20 - 49%  50 - 70%  NO_VAL  poniżej 20%  powyżej 70%
corr                                                         
0               2        18      33           12            7
1              10        48      75           18           23
Wartość statyst

In [60]:
# matura - hist - roz
test_parametr(dfs, names, 'corr', 'HIST_roz')

-----------------------------------------------
Zsumowany wynik:
HIST_roz  20 - 49%  NO_VAL  powyżej 70%
corr                                   
0                0       1            0
2                0       3            0
3                0       8            0
4                1       7            0
5                0      12            2
6                0       6            1
Wartość statystyki chi-kwadrat: 7.163127413127413
P-value: 0.7099602106772773
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
HIST_roz  20 - 49%  NO_VAL  powyżej 70%
corr                                   
0                2      68            2
1                4     154           16
Wartość statystyki chi-kwadrat: 3.1134582858720794
P-value: 0.2108245215512082
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
HIST_roz  20 - 49%  NO_VAL  powyżej 70%
corr                                   
0                1      35    

In [61]:
# matura - fiz - roz
test_parametr(dfs, names, 'corr', 'FIZ_roz')

-----------------------------------------------
Zsumowany wynik:
FIZ_roz  NO_VAL
corr           
0             1
2             3
3             8
4             8
5            14
6             7
Wartość statystyki chi-kwadrat: 0.0
P-value: 1.0
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
FIZ_roz  NO_VAL
corr           
0            72
1           174
Wartość statystyki chi-kwadrat: 0.0
P-value: 1.0
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
FIZ_roz  NO_VAL
corr           
0            36
1            87
Wartość statystyki chi-kwadrat: 0.0
P-value: 1.0
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki B:
FIZ_roz  NO_VAL
corr           
0            36
1            87
Wartość statystyki chi-kwadrat: 0.0
P-value: 1.0
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki 1:
FIZ_roz  NO_VAL
corr           
0             8
1  

In [62]:
# samopoczucie
test_parametr(dfs, names, 'corr', 'Samopocz')

-----------------------------------------------
Zsumowany wynik:
Samopocz  2  3  4  5
corr                
0         1  0  0  0
2         1  1  0  1
3         1  3  2  2
4         0  0  3  5
5         2  2  9  1
6         0  0  7  0
Wartość statystyki chi-kwadrat: 31.711857520786097
P-value: 0.007045137302814013
Istnieją podstawy do odrzucenia H0, przyjmujemy hipotezę H1:
-----------------------------------------------
Wszystkie grafiki:
Samopocz   2   3    4   5
corr                     
0         15  15   21  21
1         15  21  105  33
Wartość statystyki chi-kwadrat: 20.98108237547892
P-value: 0.00010623282905172462
Istnieją podstawy do odrzucenia H0, przyjmujemy hipotezę H1:
-----------------------------------------------
Grafiki A:
Samopocz  2  3   4   5
corr                  
0         9  9   8  10
1         6  9  55  17
Wartość statystyki chi-kwadrat: 19.722725475886392
P-value: 0.00019374713580511398
Istnieją podstawy do odrzucenia H0, przyjmujemy hipotezę H1:
----------------

In [63]:
test_posthoc(df_wynik, 'corr', 'Samopocz')
test_posthoc(df_all, 'corr', 'Samopocz')
test_posthoc(df_a, 'corr', 'Samopocz')
test_posthoc(df_3, 'corr', 'Samopocz')

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     2      3      0.5 0.8909 -1.3839  2.3839  False
     2      4      2.0 0.0069  0.4519  3.5481   True
     2      5   0.6667 0.7312 -1.0686   2.402  False
     3      4      1.5 0.0385  0.0598  2.9402   True
     3      5   0.1667 0.9927  -1.473  1.8064  False
     4      5  -1.3333 0.0309 -2.5728 -0.0938   True
----------------------------------------------------
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     2      3   0.0833 0.8687 -0.1972  0.3639  False
     2      4   0.3333 0.0013  0.1028  0.5639   True
     2      5   0.1111 0.6824 -0.1473  0.3696  False
     3      4     0.25  0.015  0.0355  0.4645   True
     3      5   0.0278 0.9911 -0.2164   0.272  False
     4      5  -0.2222 0.0111 -0.4068 -0.0376 

In [64]:
# zmęczenie
test_parametr(dfs, names, 'corr', 'Zmęcz')

-----------------------------------------------
Zsumowany wynik:
Zmęcz  1  2  3  4  5
corr                
0      0  0  0  1  0
2      2  0  0  1  0
3      0  2  3  1  2
4      1  4  2  1  0
5      3  3  5  2  1
6      0  3  1  3  0
Wartość statystyki chi-kwadrat: 23.915434102934103
P-value: 0.24610570903881462
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Zmęcz   1   2   3   4   5
corr                     
0      13  17  18  17   7
1      23  55  48  37  11
Wartość statystyki chi-kwadrat: 2.986806523085259
P-value: 0.5600357194559189
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Zmęcz   1   2   3   4  5
corr                    
0       6   8  10   8  4
1      12  28  23  19  5
Wartość statystyki chi-kwadrat: 2.0270714810944694
P-value: 0.7307795127044739
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki B:
Zmęcz   1   2   3   4  5
corr                   

In [65]:
# czytanie map
test_parametr(dfs, names, 'corr', 'Czyt_map')

-----------------------------------------------
Zsumowany wynik:
Czyt_map  3  4  5
corr             
0         0  1  0
2         1  0  2
3         1  7  0
4         1  5  2
5         1  7  6
6         1  5  1
Wartość statystyki chi-kwadrat: 10.8978354978355
P-value: 0.36553281587992115
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Czyt_map   3    4   5
corr                 
0         10   44  18
1         20  106  48
Wartość statystyki chi-kwadrat: 0.36672936259143135
P-value: 0.8324645161362327
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Czyt_map   3   4   5
corr                
0          5  22   9
1         10  53  24
Wartość statystyki chi-kwadrat: 0.18336468129571568
P-value: 0.9123949342999624
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki B:
Czyt_map   3   4   5
corr                
0          5  22   9
1         10  53  24
Wartość statystyki 

In [66]:
# podzielność uwagi
test_parametr(dfs, names, 'corr', 'Podziel_uwg')

-----------------------------------------------
Zsumowany wynik:
Podziel_uwg  1  2  3  4  5
corr                      
0            0  0  1  0  0
2            0  1  1  1  0
3            0  1  3  2  2
4            1  2  3  1  1
5            1  3  3  4  3
6            0  1  3  1  2
Wartość statystyki chi-kwadrat: 7.756259448223734
P-value: 0.9933613989615059
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Podziel_uwg  1   2   3   4   5
corr                          
0            3  14  28  16  11
1            9  34  56  38  37
Wartość statystyki chi-kwadrat: 1.7151513942103016
P-value: 0.7879635487048399
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Podziel_uwg  1   2   3   4   5
corr                          
0            1   7  16   8   4
1            5  17  26  19  20
Wartość statystyki chi-kwadrat: 3.8838012376087088
P-value: 0.42195999106490467
Brak podstaw do odrzucenia H0.
---------------

In [67]:
# analityczne myślenie
test_parametr(dfs, names, 'corr', 'Analit_myśl')

-----------------------------------------------
Zsumowany wynik:
Analit_myśl  2  3  4  5
corr                   
0            0  0  1  0
2            0  2  0  1
3            1  0  6  1
4            0  4  4  0
5            0  3  6  5
6            0  3  2  2
Wartość statystyki chi-kwadrat: 17.89539821776664
P-value: 0.2682047319931148
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Analit_myśl  2   3   4   5
corr                      
0            3  19  38  12
1            3  53  76  42
Wartość statystyki chi-kwadrat: 3.7390245849297576
P-value: 0.29105870299635156
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Analit_myśl  2   3   4   5
corr                      
0            2  10  20   4
1            1  26  37  23
Wartość statystyki chi-kwadrat: 5.7224630861956935
P-value: 0.1259220286413105
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki B:
Analit_myśl 

In [68]:
# spostrzegawczość
test_parametr(dfs, names, 'corr', 'Spostrzeg')

-----------------------------------------------
Zsumowany wynik:
Spostrzeg  2  3  4  5
corr                 
0          0  0  0  1
2          0  1  0  2
3          1  4  2  1
4          0  2  5  1
5          2  1  8  3
6          1  3  3  0
Wartość statystyki chi-kwadrat: 18.425595238095237
P-value: 0.24095198190252065
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Spostrzeg   2   3   4   5
corr                     
0           5  21  24  22
1          19  45  84  26
Wartość statystyki chi-kwadrat: 9.984467520027865
P-value: 0.018698629712990897
Istnieją podstawy do odrzucenia H0, przyjmujemy hipotezę H1:
-----------------------------------------------
Grafiki A:
Spostrzeg   2   3   4   5
corr                     
0           2  11  12  11
1          10  22  42  13
Wartość statystyki chi-kwadrat: 5.6600814176245215
P-value: 0.12937138878510396
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki B:
Spo

In [69]:
test_posthoc(df_all, 'corr', 'Spostrzeg')
test_posthoc(df_2, 'corr', 'Spostrzeg')

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     2      3  -0.1098 0.7346 -0.3869  0.1672  False
     2      4  -0.0139 0.9991 -0.2762  0.2484  False
     2      5    -0.25 0.1193 -0.5406  0.0406  False
     3      4    0.096 0.5214 -0.0857  0.2776  False
     3      5  -0.1402 0.3558 -0.3607  0.0804  False
     4      5  -0.2361 0.0144 -0.4378 -0.0345   True
----------------------------------------------------
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     2      3   0.0227 0.9993 -0.4516  0.4971  False
     2      4   0.0556 0.9881 -0.3935  0.5046  False
     2      5  -0.3125 0.3577   -0.81   0.185  False
     3      4   0.0328 0.9925 -0.2781  0.3438  False
     3      5  -0.3352 0.0997 -0.7127  0.0423  False
     4      5  -0.3681 0.0321 -0.7133 -0.0228 

In [70]:
# czytanie ze zrozumieniem
test_parametr(dfs, names, 'corr', 'Czyt_ze_zroz')

-----------------------------------------------
Zsumowany wynik:
Czyt_ze_zroz  1  2  3  4  5
corr                       
0             0  0  0  1  0
2             0  1  1  0  1
3             1  2  2  2  1
4             0  2  0  4  2
5             1  2  2  6  3
6             0  1  1  4  1
Wartość statystyki chi-kwadrat: 9.773269724556489
P-value: 0.9720992822107443
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Czyt_ze_zroz  1   2   3   4   5
corr                           
0             4  16  12  26  14
1             8  32  24  76  34
Wartość statystyki chi-kwadrat: 1.46981350011269
P-value: 0.8319748354538102
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Czyt_ze_zroz  1   2   3   4   5
corr                           
0             2   8   7  11   8
1             4  16  11  40  16
Wartość statystyki chi-kwadrat: 2.6962944181504036
P-value: 0.609863197379616
Brak podstaw do odrzucenia H0.
---

In [71]:
# wzrokowiec
test_parametr(dfs, names, 'corr', 'Wzrokowiec')

-----------------------------------------------
Zsumowany wynik:
Wzrokowiec  2  3  4  5
corr                  
0           0  0  1  0
2           0  0  1  2
3           0  2  3  3
4           1  2  3  2
5           0  3  4  7
6           1  2  3  1
Wartość statystyki chi-kwadrat: 9.054166666666665
P-value: 0.8746682389600227
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Wzrokowiec   2   3   4   5
corr                      
0            2  13  29  28
1           10  41  61  62
Wartość statystyki chi-kwadrat: 2.1512345679012346
P-value: 0.5416173965281323
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Wzrokowiec  2   3   4   5
corr                     
0           1   6  14  15
1           5  21  31  30
Wartość statystyki chi-kwadrat: 1.5407726692209454
P-value: 0.6728924893506323
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki B:
Wzrokowiec  2   3   4   5

In [72]:
# słuchowiec
test_parametr(dfs, names, 'corr', 'Słuchowiec')

-----------------------------------------------
Zsumowany wynik:
Słuchowiec  1  2  3  4  5
corr                     
0           0  0  0  0  1
2           1  1  1  0  0
3           1  2  3  2  0
4           1  1  2  1  3
5           0  4  6  2  2
6           1  1  3  2  0
Wartość statystyki chi-kwadrat: 17.716307634164778
P-value: 0.6060902624084501
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Słuchowiec   1   2   3   4   5
corr                          
0            9  16  23  10  14
1           15  38  67  32  22
Wartość statystyki chi-kwadrat: 3.6022893480508427
P-value: 0.4624963896449312
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Słuchowiec  1   2   3   4   5
corr                         
0           4   9  12   5   6
1           8  18  33  16  12
Wartość statystyki chi-kwadrat: 0.9043787629994526
P-value: 0.9239317759928483
Brak podstaw do odrzucenia H0.
---------------------------

In [73]:
test_posthoc(df_2, 'corr', 'Słuchowiec')

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     1      2   0.6389 0.0059  0.1354 1.1423   True
     1      3     0.55  0.014  0.0786 1.0214   True
     1      4   0.4643 0.1083 -0.0608 0.9894  False
     1      5   0.3333 0.4269 -0.2075 0.8741  False
     2      3  -0.0889 0.9552 -0.4421 0.2644  False
     2      4  -0.1746 0.7765 -0.5968 0.2476  False
     2      5  -0.3056 0.3089 -0.7471  0.136  False
     3      4  -0.0857 0.9707 -0.4692 0.2978  False
     3      5  -0.2167 0.5685 -0.6214  0.188  False
     4      5   -0.131 0.9343 -0.5971 0.3351  False
---------------------------------------------------


In [74]:
# kinestetyk
test_parametr(dfs, names, 'corr', 'Kinestetyk')

-----------------------------------------------
Zsumowany wynik:
Kinestetyk  1  2  3  4  5
corr                     
0           0  1  0  0  0
2           1  0  0  2  0
3           3  3  0  2  0
4           1  2  1  3  1
5           3  7  2  2  0
6           2  4  0  1  0
Wartość statystyki chi-kwadrat: 15.379306722689075
P-value: 0.7543078440196341
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Kinestetyk   1   2   3   4  5
corr                         
0           18  26   4  22  2
1           42  76  14  38  4
Wartość statystyki chi-kwadrat: 2.784771993088423
P-value: 0.5944641879049541
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Kinestetyk   1   2  3   4  5
corr                        
0            8  13  3  11  1
1           22  38  6  19  2
Wartość statystyki chi-kwadrat: 1.3387142213207124
P-value: 0.854773701210176
Brak podstaw do odrzucenia H0.
-------------------------------------

In [75]:
# MAT - pisał / nie pisał matury rozszerzonej
test_parametr(dfs, names, 'corr', 'MAT')

-----------------------------------------------
Zsumowany wynik:
MAT   NO_VAL  VAL
corr             
0          1    0
2          1    2
3          3    5
4          4    4
5          6    8
6          3    4
Wartość statystyki chi-kwadrat: 1.6770905221992185
P-value: 0.8917732287271253
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
MAT   NO_VAL  VAL
corr             
0         33   39
1         75   99
Wartość statystyki chi-kwadrat: 0.06318860245803029
P-value: 0.8015254063424021
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
MAT   NO_VAL  VAL
corr             
0         17   19
1         37   50
Wartość statystyki chi-kwadrat: 0.07705001665833755
P-value: 0.7813356714061968
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki B:
MAT   NO_VAL  VAL
corr             
0         16   20
1         38   49
Wartość statystyki chi-kwadrat: 0.0
P-value: 1.0
Brak pods

In [76]:
# POL - pisał / nie pisał matury rozszerzonej
test_parametr(dfs, names, 'corr', 'POL')

-----------------------------------------------
Zsumowany wynik:
POL   NO_VAL  VAL
corr             
0          1    0
2          3    0
3          6    2
4          5    3
5         10    4
6          7    0
Wartość statystyki chi-kwadrat: 4.624193948412698
P-value: 0.46344058010173084
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
POL   NO_VAL  VAL
corr             
0         56   16
1        136   38
Wartość statystyki chi-kwadrat: 0.0
P-value: 1.0
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
POL   NO_VAL  VAL
corr             
0         27    9
1         69   18
Wartość statystyki chi-kwadrat: 0.08185082082801189
P-value: 0.7748047294825963
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki B:
POL   NO_VAL  VAL
corr             
0         29    7
1         67   20
Wartość statystyki chi-kwadrat: 0.03712434147509581
P-value: 0.8472121097197656
Brak pods

In [77]:
# HIST - pisał / nie pisał matury rozszerzonej
test_parametr(dfs, names, 'corr', 'HIST')

-----------------------------------------------
Zsumowany wynik:
HIST  NO_VAL  VAL
corr             
0          1    0
2          3    0
3          8    0
4          7    1
5         12    2
6          6    1
Wartość statystyki chi-kwadrat: 1.8550916988416986
P-value: 0.8688104858884139
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
HIST  NO_VAL  VAL
corr             
0         68    4
1        154   20
Wartość statystyki chi-kwadrat: 1.4212561160298232
P-value: 0.23319676316900462
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
HIST  NO_VAL  VAL
corr             
0         35    1
1         76   11
Wartość statystyki chi-kwadrat: 1.80604904473439
P-value: 0.17898291181029136
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki B:
HIST  NO_VAL  VAL
corr             
0         33    3
1         78    9
Wartość statystyki chi-kwadrat: 6.633788961375382e-05
P-valu

In [78]:
# dominujący typ uczenia poznawczego
test_parametr(dfs, names, 'corr', 'Typ_uczenia')

-----------------------------------------------
Zsumowany wynik:
Typ_uczenia  Brak dominującego  Kinestetyk  Słuchowiec  Wzrokowiec
corr                                                              
0                            0           0           1           0
2                            1           0           0           2
3                            1           0           1           6
4                            2           2           2           2
5                            3           0           2           9
6                            3           0           0           4
Wartość statystyki chi-kwadrat: 19.820910973084885
P-value: 0.1789109382341189
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Typ_uczenia  Brak dominującego  Kinestetyk  Słuchowiec  Wzrokowiec
corr                                                              
0                           14           4          15          39
1                    

In [79]:
test_posthoc(df_2, 'corr', 'Typ_uczenia')

        Multiple Comparison of Means - Tukey HSD, FWER=0.05        
      group1        group2   meandiff p-adj   lower   upper  reject
-------------------------------------------------------------------
Brak dominującego Kinestetyk    -0.35 0.4633 -0.9771  0.2771  False
Brak dominującego Słuchowiec  -0.4333 0.0393 -0.8514 -0.0153   True
Brak dominującego Wzrokowiec  -0.0891 0.8708 -0.3958  0.2175  False
       Kinestetyk Słuchowiec  -0.0833 0.9874 -0.7444  0.5777  False
       Kinestetyk Wzrokowiec   0.2609 0.6615  -0.336  0.8577  False
       Słuchowiec Wzrokowiec   0.3442 0.0789 -0.0269  0.7153  False
-------------------------------------------------------------------


Podsumowanie - poprawność:
Istotne statystycznie:
- rok urodzenia: dla grafik 1 (2001 vs 2003 oraz 2001 vs 2002 --> na korzyść 2003, 2002, generalnie młodsi lepiej)
- kierunek studiów: dla grafik 3 (geografia vs geografia w ramach MISMap --> na korzyść MISMaP)
- ilość snu: dla grafik 2 (bardzo słabo się różnicuje w postHoc)
- matura podstawowa j.pol: wszystkie grafiki (30 - 49% vs 70 - 89%); generalnie lepsze wyniki matury skutkowały większym prawdopodobieństwem poprawnej odpowiedzi
- matura podstawowa mat: grafiki 3 (bardzo słabo się różnicuje w postHoc), ale generalnie lepsze wyniki matury skutkowały większym prawdopodobieństwem poprawnej odpowiedzi
- samopoczucie: zsumowane grafiki, grafiki wszystkie, grafiki A, grafiki 3 --> test postHoc wykazuje że różnice istotne statystycznie nie są dla skrajnych wartości sampoczucia, ale dla sąsiednich np. 3-4 
- spostrzegawczość: wszystkie grafiki, grafiki 2 (co się super zgadza bo to zadanie było proste, ale trzeba było to zauważyć!!!)
- słuchowiec: grafiki 2
- dominujący typ uczenia poznawczego: grafiki 2 (brak dominującego vs słuchowiec)


Trudność/pewność:

In [80]:
# płeć
test_parametr(dfs, names, 'trud', 'Płeć')

-----------------------------------------------
Zsumowany wynik:
Płeć  K  M
trud      
9     0  1
10    2  2
11    1  2
12    1  1
13    2  3
14    5  1
15    1  2
16    3  2
17    3  0
18    1  1
19    1  2
20    2  1
22    1  0
Wartość statystyki chi-kwadrat: 8.922946859903382
P-value: 0.709497338911195
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Płeć   K   M
trud        
1     23  29
2     47  37
3     41  22
4     23  15
5      4   5
Wartość statystyki chi-kwadrat: 5.836529109221712
P-value: 0.21169351918840076
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Płeć   K   M
trud        
1      9  13
2     27  16
3     21  14
4     11   9
5      1   2
Wartość statystyki chi-kwadrat: 3.7003228271934043
P-value: 0.4480789723343406
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki B:
Płeć   K   M
trud        
1     14  16
2     20  21
3     20   8
4     12  

In [81]:
# rok urodzenia
test_parametr(dfs, names, 'trud', 'Rok_urodzenia')

-----------------------------------------------
Zsumowany wynik:
Rok_urodzenia  2000  2001  2002  2003  2004
trud                                       
9                 0     1     0     0     0
10                0     1     0     3     0
11                0     0     0     3     0
12                0     0     0     2     0
13                0     0     2     2     1
14                0     2     3     1     0
15                0     0     1     1     1
16                1     1     1     1     1
17                0     1     1     1     0
18                0     2     0     0     0
19                0     0     1     2     0
20                0     0     2     1     0
22                0     0     0     1     0
Wartość statystyki chi-kwadrat: 44.80751262626262
P-value: 0.6044405428381208
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Rok_urodzenia  2000  2001  2002  2003  2004
trud                                       
1          

In [82]:
# kierunek studiów
test_parametr(dfs, names, 'trud', 'Kierunek_studiów')

-----------------------------------------------
Zsumowany wynik:
Kierunek_studiów  Geografia  Geografia, w ramach studiów międzyobszarowych  \
trud                                                                         
9                         1                                              0   
10                        4                                              0   
11                        3                                              0   
12                        2                                              0   
13                        3                                              0   
14                        4                                              1   
15                        2                                              0   
16                        4                                              0   
17                        3                                              0   
18                        2                                              0   

In [88]:
# rok studiów
test_parametr(dfs, names, 'trud', 'Rok_studiów')

-----------------------------------------------
Zsumowany wynik:
Rok_studiów  1  2  3
trud                
9            0  0  1
10           3  0  1
11           3  0  0
12           2  0  0
13           4  1  0
14           2  2  2
15           2  1  0
16           2  1  2
17           2  0  1
18           0  0  2
19           2  1  0
20           1  2  0
22           1  0  0
Wartość statystyki chi-kwadrat: 27.19097222222222
P-value: 0.29563035098757423
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Rok_studiów   1   2   3
trud                   
1            36   6  10
2            48  16  20
3            33  14  16
4            23   9   6
5             4   3   2
Wartość statystyki chi-kwadrat: 6.2493243436225905
P-value: 0.6193259572389551
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Rok_studiów   1  2   3
trud                  
1            16  4   2
2            26  7  10
3            1

In [86]:
# specjalność
test_parametr(dfs, names, 'trud', 'Specjalność')

-----------------------------------------------
Zsumowany wynik:
Specjalność  FIZ   GEOINF   NO_VAL  SPOŁ-EKO
trud                                        
9               0        1       0         0
10              1        1       0         2
11              1        2       0         0
12              1        1       0         0
13              2        1       2         0
14              0        4       1         1
15              1        1       1         0
16              1        2       1         1
17              1        2       0         0
18              0        2       0         0
19              2        0       0         1
20              0        1       0         2
22              0        0       0         1
Wartość statystyki chi-kwadrat: 33.198611111111106
P-value: 0.6025422602824994
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Specjalność  FIZ   GEOINF   NO_VAL  SPOŁ-EKO
trud                                  

In [87]:
# wady wzroku
test_parametr(dfs, names, 'trud', 'Wady_wzroku')

-----------------------------------------------
Zsumowany wynik:
Wady_wzroku  0  1
trud             
9            1  0
10           3  1
11           1  2
12           1  1
13           1  4
14           2  4
15           0  3
16           3  2
17           1  2
18           1  1
19           1  2
20           1  2
22           0  1
Wartość statystyki chi-kwadrat: 8.430625
P-value: 0.7506384917250771
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Wady_wzroku   0   1
trud               
1            24  28
2            34  50
3            22  41
4            13  25
5             3   6
Wartość statystyki chi-kwadrat: 2.123612798020693
P-value: 0.7130356158360381
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Wady_wzroku   0   1
trud               
1             8  14
2            19  24
3            12  23
4             8  12
5             1   2
Wartość statystyki chi-kwadrat: 0.9260270814456861

In [89]:
# ilość snu
test_parametr(dfs, names, 'trud', 'Ilość_snu')

-----------------------------------------------
Zsumowany wynik:
Ilość_snu  6  7  8  8<  <6
trud                      
9          0  0  1   0   0
10         2  1  0   0   1
11         0  0  0   2   1
12         0  0  0   2   0
13         0  1  1   1   2
14         2  3  0   0   1
15         1  1  0   0   1
16         1  0  1   0   3
17         1  1  0   1   0
18         1  0  1   0   0
19         3  0  0   0   0
20         1  2  0   0   0
22         0  0  0   0   1
Wartość statystyki chi-kwadrat: 63.846111111111114
P-value: 0.06255213317818922
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Ilość_snu   6   7  8  8<  <6
trud                        
1          17   5  6   9  15
2          17  24  7  16  20
3          15  17  9   9  13
4          20   7  1   2   8
5           3   1  1   0   4
Wartość statystyki chi-kwadrat: 28.128034205885083
P-value: 0.0305239435093948
Istnieją podstawy do odrzucenia H0, przyjmujemy hipotezę H1:
---------

In [90]:
test_posthoc(df_all, 'trud', 'Ilość_snu')
test_posthoc(df_3, 'trud', 'Ilość_snu')

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     6      7  -0.1157 0.9767  -0.656 0.4246  False
     6      8  -0.3194 0.7273 -1.0269  0.388  False
     6     8<  -0.5417 0.1109 -1.1543  0.071  False
     6     <6  -0.2194 0.7798 -0.7441 0.3052  False
     7      8  -0.2037 0.9416   -0.94 0.5326  False
     7     8<  -0.4259 0.3686 -1.0717 0.2199  False
     7     <6  -0.1037 0.9867 -0.6667 0.4593  False
     8     8<  -0.2222 0.9384 -1.0131 0.5687  False
     8     <6      0.1 0.9956 -0.6249 0.8249  False
    8<     <6   0.3222 0.6285 -0.3105  0.955  False
---------------------------------------------------
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     6      7   0.0417    1.0 -0.9435 1.0268  False
     6      8  -0.0833 0.9998 -1.3732 1.2065  False
     6     8

In [91]:
# matura - j.pol - podst
test_parametr(dfs, names, 'trud', 'JP_podst')

-----------------------------------------------
Zsumowany wynik:
JP_podst  30 - 49%  50 - 69%  70 - 89%  90 - 100%
trud                                             
9                0         1         0          0
10               1         0         3          0
11               1         2         0          0
12               1         0         1          0
13               0         2         3          0
14               0         1         4          1
15               0         0         2          1
16               1         2         2          0
17               1         2         0          0
18               0         1         1          0
19               0         2         0          1
20               0         1         0          2
22               0         0         1          0
Wartość statystyki chi-kwadrat: 38.44812324929973
P-value: 0.3592522773444288
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
JP_podst 

In [92]:
# matura - mat - podst
test_parametr(dfs, names, 'trud', 'MAT_podst')

-----------------------------------------------
Zsumowany wynik:
MAT_podst  30 - 49%  50 - 69%  70 - 89%  90 - 100%
trud                                              
9                 0         0         1          0
10                0         0         1          3
11                0         1         1          1
12                0         0         2          0
13                0         0         1          4
14                0         1         2          3
15                0         0         0          3
16                0         1         2          2
17                0         0         1          2
18                0         0         0          2
19                0         0         0          3
20                1         0         1          1
22                0         0         0          1
Wartość statystyki chi-kwadrat: 33.198611111111106
P-value: 0.6025422602824994
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie gr

In [93]:
# matura - j.pol - roz
test_parametr(dfs, names, 'trud', 'JP_roz')

-----------------------------------------------
Zsumowany wynik:
JP_roz  20 - 49%  50 - 70%  NO_VAL  powyżej 70%
trud                                           
9              0         0       1            0
10             1         0       3            0
11             0         0       3            0
12             0         0       2            0
13             0         1       4            0
14             0         1       3            2
15             0         0       3            0
16             0         1       4            0
17             0         0       3            0
18             0         0       2            0
19             0         0       2            1
20             0         1       2            0
22             0         1       0            0
Wartość statystyki chi-kwadrat: 32.85552083333333
P-value: 0.6189413931692247
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
JP_roz  20 - 49%  50 - 70%  NO_VAL  pow

In [94]:
# matura - mat - roz
test_parametr(dfs, names, 'trud', 'MAT_roz')

-----------------------------------------------
Zsumowany wynik:
MAT_roz  20 - 49%  50 - 70%  NO_VAL  poniżej 20%  powyżej 70%
trud                                                         
9               0         1       0            0            0
10              0         0       2            1            1
11              0         0       1            2            0
12              0         0       1            1            0
13              1         1       2            0            1
14              1         1       3            0            1
15              0         2       1            0            0
16              0         1       3            1            0
17              0         0       3            0            0
18              0         2       0            0            0
19              0         3       0            0            0
20              0         0       1            0            2
22              0         0       1            0            0
Warto

In [95]:
# matura - hist - roz
test_parametr(dfs, names, 'trud', 'HIST_roz')

-----------------------------------------------
Zsumowany wynik:
HIST_roz  20 - 49%  NO_VAL  powyżej 70%
trud                                   
9                0       1            0
10               0       4            0
11               0       3            0
12               0       2            0
13               0       5            0
14               0       4            2
15               0       3            0
16               0       4            1
17               0       3            0
18               0       2            0
19               0       3            0
20               1       2            0
22               0       1            0
Wartość statystyki chi-kwadrat: 22.40840840840841
P-value: 0.5549045130804524
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
HIST_roz  20 - 49%  NO_VAL  powyżej 70%
trud                                   
1                0      50            2
2                2      72           10

In [96]:
# matura - fiz - roz
test_parametr(dfs, names, 'trud', 'FIZ_roz')

-----------------------------------------------
Zsumowany wynik:
FIZ_roz  NO_VAL
trud           
9             1
10            4
11            3
12            2
13            5
14            6
15            3
16            5
17            3
18            2
19            3
20            3
22            1
Wartość statystyki chi-kwadrat: 0.0
P-value: 1.0
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
FIZ_roz  NO_VAL
trud           
1            52
2            84
3            63
4            38
5             9
Wartość statystyki chi-kwadrat: 0.0
P-value: 1.0
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
FIZ_roz  NO_VAL
trud           
1            22
2            43
3            35
4            20
5             3
Wartość statystyki chi-kwadrat: 0.0
P-value: 1.0
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki B:
FIZ_roz  NO_VAL
trud           
1            3

In [97]:
# samopoczucie
test_parametr(dfs, names, 'trud', 'Samopocz')

-----------------------------------------------
Zsumowany wynik:
Samopocz  2  3  4  5
trud                
9         0  0  0  1
10        0  0  3  1
11        0  0  0  3
12        1  0  1  0
13        0  2  2  1
14        1  1  2  2
15        1  0  2  0
16        0  0  5  0
17        1  1  1  0
18        0  0  2  0
19        1  0  2  0
20        0  1  1  1
22        0  1  0  0
Wartość statystyki chi-kwadrat: 43.896031746031746
P-value: 0.1717128804716939
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Samopocz   2  3   4   5
trud                   
1          4  7  24  17
2         11  9  44  20
3          9  9  33  12
4          6  8  19   5
5          0  3   6   0
Wartość statystyki chi-kwadrat: 13.953288134867083
P-value: 0.3037011651537289
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Samopocz  2  3   4   5
trud                  
1         3  3   9   7
2         5  5  21  12
3         5  5

In [98]:
test_posthoc(df_2, 'trud', 'Samopocz')
test_posthoc(df_3, 'trud', 'Samopocz')

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     2      3     0.35 0.8216 -0.7093  1.4093  False
     2      4  -0.9952 0.0185 -1.8658 -0.1247   True
     2      5  -0.7333  0.207 -1.7091  0.2424  False
     3      4  -1.3452 0.0002  -2.155 -0.5354   True
     3      5  -1.0833 0.0147 -2.0053 -0.1613   True
     4      5   0.2619 0.7575 -0.4351  0.9589  False
----------------------------------------------------
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     2      3   0.2167 0.9614 -0.9471 1.3804  False
     2      4   0.6571 0.2793 -0.2992 1.6135  False
     2      5  -0.7556  0.258 -1.8275 0.3164  False
     3      4   0.4405 0.5658 -0.4492 1.3301  False
     3      5  -0.9722 0.0646 -1.9851 0.0407  False
     4      5  -1.4127    0.0 -2.1784 -0.647   True
--

In [99]:
# zmęczenie
test_parametr(dfs, names, 'trud', 'Zmęcz')

-----------------------------------------------
Zsumowany wynik:
Zmęcz  1  2  3  4  5
trud                
9      0  1  0  0  0
10     1  0  3  0  0
11     1  2  0  0  0
12     1  1  0  0  0
13     1  3  0  0  1
14     2  1  2  1  0
15     0  1  2  0  0
16     0  0  1  3  1
17     0  1  0  1  1
18     0  1  0  1  0
19     0  0  3  0  0
20     0  1  0  2  0
22     0  0  0  1  0
Wartość statystyki chi-kwadrat: 55.02558922558922
P-value: 0.2259975927255554
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Zmęcz   1   2   3   4  5
trud                    
1       8  17  15   9  3
2      19  25  21  12  7
3       8  22  17  12  4
4       1   7  12  15  3
5       0   1   1   6  1
Wartość statystyki chi-kwadrat: 30.753982128982127
P-value: 0.014463091048514723
Istnieją podstawy do odrzucenia H0, przyjmujemy hipotezę H1:
-----------------------------------------------
Grafiki A:
Zmęcz   1   2   3  4  5
trud                   
1       3   8   6  3

In [100]:
test_posthoc(df_all, 'trud', 'Zmęcz')

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     1      2     0.25 0.7812  -0.349  0.849  False
     1      3   0.3838 0.4144 -0.2242 0.9918  False
     1      4   0.8889 0.0013  0.2575 1.5203   True
     1      5      0.5 0.4846 -0.3471 1.3471  False
     2      3   0.1338  0.948 -0.3662 0.6339  False
     2      4   0.6389  0.009  0.1106 1.1671   True
     2      5     0.25 0.9009 -0.5233 1.0233  False
     3      4   0.5051 0.0777 -0.0334 1.0435  False
     3      5   0.1162 0.9941 -0.6641 0.8965  False
     4      5  -0.3889 0.6675 -1.1875 0.4098  False
---------------------------------------------------


In [101]:
# czytanie map
test_parametr(dfs, names, 'trud', 'Czyt_map')

-----------------------------------------------
Zsumowany wynik:
Czyt_map  3  4  5
trud             
9         0  0  1
10        0  3  1
11        1  1  1
12        2  0  0
13        0  5  0
14        0  4  2
15        0  1  2
16        1  3  1
17        0  3  0
18        0  1  1
19        0  2  1
20        1  1  1
22        0  1  0
Wartość statystyki chi-kwadrat: 31.378666666666664
P-value: 0.14316186976350656
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Czyt_map  3   4   5
trud               
1         8  26  18
2         9  57  18
3         8  38  17
4         4  25   9
5         1   4   4
Wartość statystyki chi-kwadrat: 6.195052549789393
P-value: 0.6253932822108781
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Czyt_map  3   4   5
trud               
1         3  12   7
2         5  28  10
3         4  21  10
4         3  12   5
5         0   2   1
Wartość statystyki chi-kwadrat: 1.37576

In [102]:
# podzielność uwagi
test_parametr(dfs, names, 'trud', 'Podziel_uwg')

-----------------------------------------------
Zsumowany wynik:
Podziel_uwg  1  2  3  4  5
trud                      
9            0  0  1  0  0
10           0  0  1  2  1
11           0  0  2  1  0
12           0  2  0  0  0
13           0  3  0  1  1
14           0  1  0  3  2
15           0  0  1  0  2
16           1  0  1  2  1
17           0  0  3  0  0
18           0  1  1  0  0
19           0  1  1  0  1
20           1  0  2  0  0
22           0  0  1  0  0
Wartość statystyki chi-kwadrat: 54.4727843915344
P-value: 0.2418247163927058
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Podziel_uwg  1   2   3   4   5
trud                          
1            1  11  17  14   9
2            5  18  21  23  17
3            1  11  25  11  15
4            3   6  17   5   7
5            2   2   4   1   0
Wartość statystyki chi-kwadrat: 20.280756805271217
P-value: 0.20784083323231248
Brak podstaw do odrzucenia H0.
---------------------------

In [103]:
test_posthoc(df_3, 'trud', 'Podziel_uwg')

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     1      2  -1.6875 0.0631 -3.4334  0.0584  False
     1      3  -1.6429 0.0559 -3.3123  0.0265  False
     1      4  -1.9444 0.0194 -3.6708 -0.2181   True
     1      5  -1.4375  0.156 -3.1834  0.3084  False
     2      3   0.0446 0.9999 -0.9341  1.0234  False
     2      4  -0.2569 0.9625   -1.33  0.8161  False
     2      5     0.25 0.9693 -0.8542  1.3542  False
     3      4  -0.3016 0.8987 -1.2451  0.6419  False
     3      5   0.2054 0.9768 -0.7734  1.1841  False
     4      5   0.5069 0.6799 -0.5661    1.58  False
----------------------------------------------------


In [105]:
# analityczne myślenie
test_parametr(dfs, names, 'trud', 'Analit_myśl')

-----------------------------------------------
Zsumowany wynik:
Analit_myśl  2  3  4  5
trud                   
9            0  0  1  0
10           0  0  4  0
11           1  1  1  0
12           0  1  1  0
13           0  3  1  1
14           0  2  3  1
15           0  1  0  2
16           0  0  1  4
17           0  0  3  0
18           0  1  1  0
19           0  1  2  0
20           0  2  0  1
22           0  0  1  0
Wartość statystyki chi-kwadrat: 45.599512670565296
P-value: 0.13113418389852613
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Analit_myśl  2   3   4   5
trud                      
1            3  13  26  10
2            1  25  41  17
3            2  19  28  14
4            0  13  15  10
5            0   2   4   3
Wartość statystyki chi-kwadrat: 6.8588992912630955
P-value: 0.8667954199474579
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Analit_myśl  2   3   4  5
trud         

In [106]:
# spostrzegawczość
test_parametr(dfs, names, 'trud', 'Spostrzeg')

-----------------------------------------------
Zsumowany wynik:
Spostrzeg  2  3  4  5
trud                 
9          0  0  1  0
10         0  1  1  2
11         0  0  1  2
12         1  1  0  0
13         0  2  2  1
14         0  1  4  1
15         0  1  1  1
16         1  2  2  0
17         0  1  1  1
18         1  0  1  0
19         0  1  2  0
20         0  1  2  0
22         1  0  0  0
Wartość statystyki chi-kwadrat: 35.210648148148145
P-value: 0.5059445822994364
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Spostrzeg  2   3   4   5
trud                    
1          4  13  19  16
2          8  22  37  17
3          4  17  33   9
4          4  13  15   6
5          4   1   4   0
Wartość statystyki chi-kwadrat: 21.935591597324052
P-value: 0.03824805030966442
Istnieją podstawy do odrzucenia H0, przyjmujemy hipotezę H1:
-----------------------------------------------
Grafiki A:
Spostrzeg  2   3   4  5
trud                   
1    

In [107]:
test_posthoc(df_all, 'trud', 'Spostrzeg')
test_posthoc(df_a, 'trud', 'Spostrzeg')
test_posthoc(df_3, 'trud', 'Spostrzeg')

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     2      3  -0.3333 0.5715 -1.0028 0.3362  False
     2      4  -0.3148 0.5734 -0.9487  0.319  False
     2      5  -0.7292 0.0384 -1.4314 -0.027   True
     3      4   0.0185 0.9995 -0.4203 0.4574  False
     3      5  -0.3958 0.2216 -0.9286  0.137  False
     4      5  -0.4144 0.1261 -0.9016 0.0729  False
---------------------------------------------------
Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     2      3  -0.6818  0.205 -1.5834  0.2198  False
     2      4  -0.6667  0.181 -1.5202  0.1869  False
     2      5  -0.9583 0.0457 -1.9039 -0.0128   True
     3      4   0.0152 0.9999 -0.5758  0.6061  False
     3      5  -0.2765 0.7472  -0.994   0.441  False
     4      5  -0.2917 0.6542 -0.9478  0.3645  False
---

In [108]:
# czytanie ze zrozumieniem
test_parametr(dfs, names, 'trud', 'Czyt_ze_zroz')

-----------------------------------------------
Zsumowany wynik:
Czyt_ze_zroz  1  2  3  4  5
trud                       
9             0  0  0  1  0
10            0  1  1  2  0
11            0  2  0  0  1
12            0  0  1  1  0
13            1  2  0  2  0
14            0  0  2  2  2
15            0  1  0  0  2
16            0  0  0  4  1
17            0  0  1  2  0
18            0  0  1  1  0
19            0  1  0  1  1
20            0  1  0  1  1
22            1  0  0  0  0
Wartość statystyki chi-kwadrat: 56.028308823529414
P-value: 0.19908055962254195
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Czyt_ze_zroz  1   2   3   4   5
trud                           
1             2  13   4  24   9
2             3  18  17  33  13
3             3   7  13  25  15
4             2   8   2  15  11
5             2   2   0   5   0
Wartość statystyki chi-kwadrat: 23.757318250062056
P-value: 0.09494055049288855
Brak podstaw do odrzucenia H0.
--

In [109]:
# wzrokowiec
test_parametr(dfs, names, 'trud', 'Wzrokowiec')

-----------------------------------------------
Zsumowany wynik:
Wzrokowiec  2  3  4  5
trud                  
9           0  0  0  1
10          0  1  1  2
11          0  2  0  1
12          0  0  2  0
13          0  2  2  1
14          0  1  2  3
15          0  0  2  1
16          0  1  1  3
17          0  0  2  1
18          1  1  0  0
19          0  0  2  1
20          1  1  0  1
22          0  0  1  0
Wartość statystyki chi-kwadrat: 36.36851851851851
P-value: 0.451494522296548
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Wzrokowiec  2   3   4   5
trud                     
1           1  13  17  21
2           4  19  28  33
3           1  14  28  20
4           4   7  14  13
5           2   1   3   3
Wartość statystyki chi-kwadrat: 13.712943862505266
P-value: 0.31941356155860456
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
Wzrokowiec  2   3   4   5
trud                     
1          

In [110]:
test_posthoc(df_1, 'trud', 'Wzrokowiec')

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
     2      3  -0.9444  0.269 -2.3022 0.4133  False
     2      4  -0.7333 0.4589 -2.0408 0.5741  False
     2      5  -0.8667 0.3102 -2.1741 0.4408  False
     3      4   0.2111 0.8735 -0.5212 0.9434  False
     3      5   0.0778 0.9924 -0.6545 0.8101  False
     4      5  -0.1333 0.9458 -0.7675 0.5009  False
---------------------------------------------------


In [111]:
# słuchowiec
test_parametr(dfs, names, 'trud', 'Słuchowiec')

-----------------------------------------------
Zsumowany wynik:
Słuchowiec  1  2  3  4  5
trud                     
9           0  0  1  0  0
10          0  1  2  1  0
11          1  0  0  2  0
12          0  1  1  0  0
13          0  1  3  1  0
14          0  1  2  1  2
15          0  0  2  0  1
16          2  2  0  1  0
17          0  1  1  0  1
18          0  0  1  1  0
19          1  2  0  0  0
20          0  0  1  0  2
22          0  0  1  0  0
Wartość statystyki chi-kwadrat: 47.243280423280424
P-value: 0.5037676799272037
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Słuchowiec  1   2   3   4   5
trud                         
1           5  11  22  11   3
2           7  19  29  18  11
3           6  11  24   8  14
4           5  11  12   3   7
5           1   2   3   2   1
Wartość statystyki chi-kwadrat: 13.011007014891726
P-value: 0.6719528096494183
Brak podstaw do odrzucenia H0.
-----------------------------------------------


In [112]:
# kinestetyk
test_parametr(dfs, names, 'trud', 'Kinestetyk')

-----------------------------------------------
Zsumowany wynik:
Kinestetyk  1  2  3  4  5
trud                     
9           0  0  0  1  0
10          1  2  0  1  0
11          0  1  0  2  0
12          1  0  0  1  0
13          1  1  1  2  0
14          2  4  0  0  0
15          0  3  0  0  0
16          2  3  0  0  0
17          0  2  0  1  0
18          1  0  0  1  0
19          1  0  1  0  1
20          1  0  1  1  0
22          0  1  0  0  0
Wartość statystyki chi-kwadrat: 47.24111111111112
P-value: 0.503856843401779
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
Kinestetyk   1   2  3   4  5
trud                        
1           13  17  2  19  1
2           18  41  6  19  0
3           15  30  3  13  2
4           12  10  6   7  3
5            2   4  1   2  0
Wartość statystyki chi-kwadrat: 22.51720329607327
P-value: 0.12726284171299151
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A

In [113]:
# MAT - pisał / nie pisał matury rozszerzonej
test_parametr(dfs, names, 'trud', 'MAT')

-----------------------------------------------
Zsumowany wynik:
MAT   NO_VAL  VAL
trud             
9          0    1
10         2    2
11         1    2
12         1    1
13         2    3
14         3    3
15         1    2
16         3    2
17         3    0
18         0    2
19         0    3
20         1    2
22         1    0
Wartość statystyki chi-kwadrat: 10.953140096618355
P-value: 0.5329388152888139
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
MAT   NO_VAL  VAL
trud             
1         18   34
2         42   42
3         31   32
4         13   25
5          4    5
Wartość statystyki chi-kwadrat: 5.259213700670605
P-value: 0.261719354956213
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
MAT   NO_VAL  VAL
trud             
1          9   13
2         20   23
3         17   18
4          6   14
5          2    1
Wartość statystyki chi-kwadrat: 2.7095075897401486
P-value: 0.6075519

In [114]:
# POL - pisał / nie pisał matury rozszerzonej
test_parametr(dfs, names, 'trud', 'POL')

-----------------------------------------------
Zsumowany wynik:
POL   NO_VAL  VAL
trud             
9          1    0
10         3    1
11         3    0
12         2    0
13         4    1
14         3    3
15         3    0
16         4    1
17         3    0
18         2    0
19         2    1
20         2    1
22         0    1
Wartość statystyki chi-kwadrat: 10.745891203703703
P-value: 0.5508122714480357
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
POL   NO_VAL  VAL
trud             
1         45    7
2         63   21
3         48   15
4         31    7
5          5    4
Wartość statystyki chi-kwadrat: 5.704480440774301
P-value: 0.2223315893238313
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
POL   NO_VAL  VAL
trud             
1         20    2
2         33   10
3         26    9
4         15    5
5          2    1
Wartość statystyki chi-kwadrat: 2.791098292588118
P-value: 0.5933702

In [115]:
# HIST - pisał / nie pisał matury rozszerzonej
test_parametr(dfs, names, 'trud', 'HIST')

-----------------------------------------------
Zsumowany wynik:
HIST  NO_VAL  VAL
trud             
9          1    0
10         4    0
11         3    0
12         2    0
13         5    0
14         4    2
15         3    0
16         4    1
17         3    0
18         2    0
19         3    0
20         2    1
22         1    0
Wartość statystyki chi-kwadrat: 9.197297297297297
P-value: 0.6859926300016987
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Wszystkie grafiki:
HIST  NO_VAL  VAL
trud             
1         50    2
2         72   12
3         59    4
4         34    4
5          7    2
Wartość statystyki chi-kwadrat: 6.465156741472532
P-value: 0.16699907266830016
Brak podstaw do odrzucenia H0.
-----------------------------------------------
Grafiki A:
HIST  NO_VAL  VAL
trud             
1         21    1
2         37    6
3         34    1
4         17    3
5          2    1
Wartość statystyki chi-kwadrat: 5.949796168110121
P-value: 0.2029290

In [116]:
# dominujący typ uczenia poznawczego
test_parametr(dfs, names, 'trud', 'Typ_uczenia')

-----------------------------------------------
Zsumowany wynik:
Typ_uczenia  Brak dominującego  Kinestetyk  Słuchowiec  Wzrokowiec
trud                                                              
9                            0           0           0           1
10                           0           1           0           3
11                           1           0           1           1
12                           1           0           0           1
13                           3           0           0           2
14                           2           0           1           3
15                           0           0           1           2
16                           1           0           0           4
17                           0           0           1           2
18                           2           0           0           0
19                           0           1           0           2
20                           0           0           2          

Podsumowanie - trudność:
Istotne statystycznie:
- ilość snu: wszystkie grafiki (mało różnicuje), grafiki 3 (widoczna różnica <6 a >8)
- sampoczoucie: grafiki 2, 3 (podobne wnioski jak poprawność)
- zmęczenie: wszystkie grafiki (1 vs 4, 2 vs 4)
- podzielność uwagi: grafiki 3 (1 vs 4)
- spostrzegawczość: grafiki wszystkie, A, 3 (głównie 2 vs 5)
- wzrokowiec: grafiki 1 (słabo się różnicuje)